In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [11]:
#column = ['tweets','duygu','preprocessing']
#df = pd.read_excel("../dataset/total.xlsx")

column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")


df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [12]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


In [13]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,hayatimda verdigim en en dogru uc karar arasin...,0
1,yil 2015 oldu hala internet çekmiyor turkcell,0
2,@kaan_terzioglu yanında kalıyoruz turkcell fat...,0
3,@jasoniks ooo buram buram turkcell plaza çalış...,1
4,size 20 yıldır dünyaları ödedim ama siz platin...,0
...,...,...
9121,"turkcell'den köylere 4,5g hizmeti",1
9122,turkcell adam akıllı tek bir reklamın yok amk,0
9123,turkcell:turkiye'nin cekim gucu amerika'yi gec...,0
9124,birideçıkıpdemiyorki bu kızın televizyonu yok ...,1


In [14]:
X_t=df['tweets'].tolist()

In [15]:
# Creating a tokenizer
tokenizer = Tokenizer(lower=True)
# Building word indices
tokenizer.fit_on_texts(X_t)
# Tokenizing sentences
sentences = tokenizer.texts_to_sequences(X_t)
# Creating a reverse dictionary
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)
# Creating texts 
X_t = list(map(sequence_to_text, sentences))


In [16]:
from gensim.models import Word2Vec
import time
start=time.time()
n_features=32
window_size=5
min_count=1
epoch=50
n_workers=8

wordembeddings = Word2Vec(size = n_features,
            window = window_size, 
            min_count= min_count,
            workers = n_workers, 
            sg=1)
wordembeddings.build_vocab(X_t)
wordembeddings.train(X_t, 
            total_examples=wordembeddings.corpus_count,  
            epochs = epoch)
end=time.time()
print(end-start)

44.21199584007263


In [17]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [18]:
def word2vec (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    #tokenizer = Tokenizer(lower=True) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    skipped_words = 0
    embedding_dim = 32 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = wordembeddings[word]#wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak

        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector #dictionarydeki indexine word2vec teki sayısal hali yazılır
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)
    
    return embedding_layer,X_train,y_train,X_test,y_test

In [19]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= word2vec (X_train,y_train,X_test,y_test)
    
    modelCNN = Sequential() #cnn

    modelCNN.add(embedding_layer)
    modelCNN.add(Conv1D(filters=256, kernel_size=5, activation='relu')) #kernal size 5 yan yana beş kelimeye bakması
    modelCNN.add(MaxPooling1D(pool_size=2)) #tek satırlık 1d olduğu için
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(360, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(300, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(260, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(150, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(120, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(80, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Flatten()) #düzgünleştirmek için
    modelCNN.add(Dense(1, activation='sigmoid'))
    modelCNN.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNN.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelCNN.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

<ipython-input-18-7d0886ed6bc1>:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  embedding_vector = wordembeddings[word]#wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 16s 63ms/step - loss: 0.5759 - acc: 0.6924 - precision_2: 0.7038 - recall_2: 0.6640
Epoch 2/50
257/257 [==============================] - 15s 58ms/step - loss: 0.4828 - acc: 0.7726 - precision_2: 0.7808 - recall_2: 0.7576
Epoch 3/50
257/257 [==============================] - 15s 58ms/step - loss: 0.4452 - acc: 0.7924 - precision_2: 0.8048 - recall_2: 0.7717
Epoch 4/50
257/257 [==============================] - 15s 60ms/step - loss: 0.4232 - acc: 0.8085 - precision_2: 0.8168 - recall_2: 0.7951: 
Epoch 5/50
257/257 [==============================] - 15s 59ms/step - loss: 0.3880 - acc: 0.8209 - precision_2: 0.8266 - recall_2: 0.8119
Epoch 6/50
257/257 [==============================] - 15s 58ms/step - loss: 0.3592 - acc: 0.8357 - precision_2: 0.8446 - recall_2: 0.8226
Epoch 7/50
257/257 [==============================] - 15s 59ms/step - los

257/257 [==============================] - 11s 43ms/step - loss: 0.1693 - acc: 0.9332 - precision_3: 0.9383 - recall_3: 0.9276
Epoch 16/50
257/257 [==============================] - 11s 43ms/step - loss: 0.1575 - acc: 0.9406 - precision_3: 0.9420 - recall_3: 0.9392
Epoch 17/50
257/257 [==============================] - 11s 43ms/step - loss: 0.1419 - acc: 0.9422 - precision_3: 0.9430 - recall_3: 0.94145s - los - ETA: 
Epoch 18/50
257/257 [==============================] - 11s 43ms/step - loss: 0.1354 - acc: 0.9498 - precision_3: 0.9508 - recall_3: 0.9490
Epoch 19/50
257/257 [==============================] - 12s 45ms/step - loss: 0.1356 - acc: 0.9476 - precision_3: 0.9484 - recall_3: 0.9470
Epoch 20/50
257/257 [==============================] - 12s 47ms/step - loss: 0.1202 - acc: 0.9528 - precision_3: 0.9551 - recall_3: 0.9504
Epoch 21/50
257/257 [==============================] - 15s 59ms/step - loss: 0.1204 - acc: 0.9531 - precision_3: 0.9527 - recall_3: 0.9538
Epoch 22/50
257/257 [==

257/257 [==============================] - 15s 58ms/step - loss: 0.0885 - acc: 0.9664 - precision_4: 0.9675 - recall_4: 0.9647
Epoch 33/50
257/257 [==============================] - 15s 57ms/step - loss: 0.0928 - acc: 0.9649 - precision_4: 0.9640 - recall_4: 0.9654
Epoch 34/50
257/257 [==============================] - 15s 59ms/step - loss: 0.0883 - acc: 0.9671 - precision_4: 0.9683 - recall_4: 0.9654
Epoch 35/50
257/257 [==============================] - 15s 57ms/step - loss: 0.0853 - acc: 0.9691 - precision_4: 0.9693 - recall_4: 0.9684
Epoch 36/50
257/257 [==============================] - 15s 58ms/step - loss: 0.0875 - acc: 0.9658 - precision_4: 0.9652 - recall_4: 0.96594s - loss: 0.0862 - acc: 0.9648  - ETA: 2s - loss: 0.0893 - acc: 0.9640 - precision_4: 0.9
Epoch 37/50
257/257 [==============================] - 15s 57ms/step - loss: 0.0851 - acc: 0.9680 - precision_4: 0.9692 - recall_4: 0.9662
Epoch 38/50
257/257 [==============================] - 15s 58ms/step - loss: 0.0845 - ac

257/257 [==============================] - 15s 59ms/step - loss: 0.1612 - acc: 0.9350 - precision_6: 0.9334 - recall_6: 0.9370: 10s - loss: 0.1467 - acc: 0.94 - ETA: 9s - l - ETA: 7s
Epoch 16/50
257/257 [==============================] - 16s 61ms/step - loss: 0.1600 - acc: 0.9386 - precision_6: 0.9349 - recall_6: 0.94311s - loss: 0 - ETA: 0s - loss: 0.1600 - acc: 0.9387 - precision_6: 0.9350 - recall_6: 0.94
Epoch 17/50
257/257 [==============================] - 11s 43ms/step - loss: 0.0968 - acc: 0.9657 - precision_6: 0.9652 - recall_6: 0.9662
Epoch 28/50
257/257 [==============================] - 11s 44ms/step - loss: 0.0939 - acc: 0.9655 - precision_6: 0.9677 - recall_6: 0.9633
Epoch 29/50
257/257 [==============================] - 11s 43ms/step - loss: 0.0977 - acc: 0.9609 - precision_6: 0.9624 - recall_6: 0.95941s - loss: 0.0993 - acc: 0.9604 - precisio
Epoch 30/50
257/257 [==============================] - 11s 43ms/step - loss: 0.0995 - acc: 0.9641 - precision_6: 0.9642 - recall_

257/257 [==============================] - 15s 57ms/step - loss: 0.0738 - acc: 0.9724 - precision_7: 0.9730 - recall_7: 0.9718
Epoch 41/50
257/257 [==============================] - 15s 57ms/step - loss: 0.0896 - acc: 0.9660 - precision_7: 0.9659 - recall_7: 0.96620s - loss: 0.0896 - acc: 0.9660 - precision_7: 0.9664 - re
Epoch 42/50
257/257 [==============================] - 11s 44ms/step - loss: 0.0704 - acc: 0.9728 - precision_7: 0.9721 - recall_7: 0.9737: 13s - loss: 0.0473 - acc: 0.9861 - precisi - ETA: 1s - loss: 0.0681 - acc: 0.9738 - precision_
Epoch 50/50
29/29 [==============================] - 0s 16ms/step - loss: 1.1769 - acc: 0.7678 - precision_7: 0.7814 - recall_7: 0.7401
------------------------------------------------------------------------
Training for fold 7 ...
Epoch 1/50
257/257 [==============================] - 15s 56ms/step - loss: 0.6053 - acc: 0.6575 - precision_8: 0.6605 - recall_8: 0.64591s - loss: 0.6155 - acc: 0.6482 - precision_8: 0.6477 - reca - ETA: 1s 

257/257 [==============================] - 11s 45ms/step - loss: 0.2334 - acc: 0.9009 - precision_9: 0.9053 - recall_9: 0.89474s - loss: 0.2340  - ETA: 2s - loss: 0
Epoch 12/50
257/257 [==============================] - 11s 44ms/step - loss: 0.2197 - acc: 0.9042 - precision_9: 0.9032 - recall_9: 0.90472s - loss:
Epoch 13/50
257/257 [==============================] - 11s 44ms/step - loss: 0.2044 - acc: 0.9145 - precision_9: 0.9167 - recall_9: 0.91136s - loss: 0.199 - ETA: 4s - loss: 0.2043 - acc: 0.9156 - precision_9: 0.9186 - recall_9: 0. - ETA: 4s - loss: 0.2038 - acc: 0.9158 - precision_9: - ETA: 3s - loss: 0.2038 - acc: 0.9144 - precision_9: 0.917 - ETA: 2s - loss:
Epoch 14/50
257/257 [==============================] - 11s 43ms/step - loss: 0.1851 - acc: 0.9268 - precision_9: 0.9290 - recall_9: 0.9238
Epoch 15/50
257/257 [==============================] - 11s 44ms/step - loss: 0.1734 - acc: 0.9308 - precision_9: 0.9325 - recall_9: 0.92849s - loss: 0.1580 - acc: 0.9427 - pr - ETA: 8s

257/257 [==============================] - 10s 41ms/step - loss: 0.0896 - acc: 0.9674 - precision_10: 0.9675 - recall_10: 0.9670
Epoch 28/50
257/257 [==============================] - 10s 41ms/step - loss: 0.0987 - acc: 0.9621 - precision_10: 0.9640 - recall_10: 0.9600
Epoch 29/50
257/257 [==============================] - 10s 41ms/step - loss: 0.0869 - acc: 0.9685 - precision_10: 0.9694 - recall_10: 0.9673
Epoch 30/50
257/257 [==============================] - 11s 41ms/step - loss: 0.0796 - acc: 0.9697 - precision_10: 0.9695 - recall_10: 0.9697
Epoch 31/50
257/257 [==============================] - 10s 41ms/step - loss: 0.0962 - acc: 0.9617 - precision_10: 0.9633 - recall_10: 0.9597
Epoch 32/50
257/257 [==============================] - 10s 41ms/step - loss: 0.0755 - acc: 0.9720 - precision_10: 0.9706 - recall_10: 0.9734
Epoch 33/50
257/257 [==============================] - 11s 42ms/step - loss: 0.0851 - acc: 0.9692 - precision_10: 0.9706 - recall_10: 0.96752s - l
Epoch 34/50
257/257

257/257 [==============================] - 11s 41ms/step - loss: 0.0836 - acc: 0.9688 - precision_11: 0.9710 - recall_11: 0.96685s - loss: 0.0788 - acc: 0
Epoch 45/50
257/257 [==============================] - 10s 41ms/step - loss: 0.0767 - acc: 0.9731 - precision_11: 0.9747 - recall_11: 0.97171s - loss: 0.0776 - acc: 0.9721 - precision_11
Epoch 46/50
257/257 [==============================] - 10s 40ms/step - loss: 0.0738 - acc: 0.9732 - precision_11: 0.9757 - recall_11: 0.9709
Epoch 47/50
257/257 [==============================] - 11s 41ms/step - loss: 0.0747 - acc: 0.9729 - precision_11: 0.9729 - recall_11: 0.9731
Epoch 00047: early stopping
29/29 [==============================] - 0s 10ms/step - loss: 0.8771 - acc: 0.7752 - precision_11: 0.7743 - recall_11: 0.7477


In [20]:
def Average(lst):
    return sum(lst) / len(lst)

In [21]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.761775
test precision: 0.768378
test recall: 0.751735
test f1_score: 0.758809
